In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Preprocessing

In [ ]:
import pandas as pd

In [129]:
# Read the Redfin data into a DataFrame
file_path_redfin = "../data/redfin_data.csv"
df_redfin = pd.read_csv(file_path_redfin)
df_redfin["month"] = pd.to_datetime(df_redfin["month"]).dt.to_period("M")
df_redfin.head()

,period_begin,period_end,region_name,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,median_sale_price,median_sale_price_yoy,age_of_inventory,age_of_inventory_yoy,State,month
0,2017-01-09,2017-02-05,Ulster County,34.0,0.096774,179693.7,0.060140,138.0,-11.2,NY,2017-01
1,2017-01-09,2017-02-05,Crook County,4.0,-0.200000,382300.0,0.950169,136.3,-9.5,OR,2017-01
2,2017-01-09,2017-02-05,Bradley County,26.0,0.238095,157927.0,0.101208,95.7,-20.3,TN,2017-01
3,2017-01-09,2017-02-05,Carter County,8.0,0.142857,153975.0,0.446963,89.1,-13.4,TN,2017-01
4,2017-01-09,2017-02-05,Cumberland County,12.0,0.000000,140562.5,-0.151321,124.0,-30.3,TN,2017-01


In [101]:
#Inspect Data
df_redfin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 518336 entries, 1 to 2339453
Data columns (total 11 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   period_begin                     518336 non-null  datetime64[ns]
 1   period_end                       518336 non-null  object        
 2   region_name                      518336 non-null  object        
 3   adjusted_average_homes_sold      518336 non-null  float64       
 4   adjusted_average_homes_sold_yoy  518336 non-null  float64       
 5   median_sale_price                518336 non-null  float64       
 6   median_sale_price_yoy            518336 non-null  float64       
 7   age_of_inventory                 518336 non-null  float64       
 8   age_of_inventory_yoy             518336 non-null  float64       
 9   State                            518336 non-null  object        
 10  month                            518336 non-null

In [109]:
# Read the Zillow data into a DataFrame
file_path_zillow_zori = "../data/County_zori_uc_sfrcondomfr_sm_month.csv"
file_path_zillow_zhvi = "../data/County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"
df_zillow_zori = pd.read_csv(file_path_zillow1)
df_zillow_zhvi = pd.read_csv(file_path_zillow2)

In [110]:
#Extract required features
df_zillow_zori.drop(columns=["RegionID", "SizeRank", "StateName", "RegionType", "StateCodeFIPS", "MunicipalCodeFIPS"], inplace=True)
df_zillow_zhvi.drop(columns=["RegionID", "SizeRank", "StateName", "RegionType", "StateCodeFIPS", "MunicipalCodeFIPS"], inplace=True)

#restructure so that the dates become rows
df_zillow_zori = pd.melt(df_zillow_zori, id_vars=["RegionName", "State", "Metro"],
                        var_name="Date",
                        value_name="ZORI")
df_zillow_zhvi = pd.melt(df_zillow_zhvi, id_vars=["RegionName", "State", "Metro"],
                        var_name="Date",
                        value_name="ZHVI")

#apply time horizon
df_zillow_zori["Date"] = pd.to_datetime(df_zillow_zori["Date"], infer_datetime_format = True)
df_zillow_zhvi["Date"] = pd.to_datetime(df_zillow_zhvi["Date"], infer_datetime_format = True)
df_zillow_zhvi = df_zillow_zhvi[df_zillow_zhvi["Date"] > "2015-01-31"]

#merge
df_zillow = pd.merge(df_zillow_zhvi, df_zillow_zori[["RegionName", "State", "Metro", "Date", "ZORI"]],
                     on = ["RegionName", "State", "Metro", "Date"],
                     how = "left")
df_zillow.dropna(inplace=True)

#define month
df_zillow["Date"] = pd.to_datetime(df_zillow["Date"])
df_zillow["month"] = df_zillow["Date"].dt.to_period("M")

#rename column to match df_redfin
df_zillow.rename(columns={"RegionName": "region_name"}, inplace=True)

df_zillow.head()

<ipython-input-110-30845324342b>:14: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_zillow_zori["Date"] = pd.to_datetime(df_zillow_zori["Date"], infer_datetime_format = True)
<ipython-input-110-30845324342b>:15: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_zillow_zhvi["Date"] = pd.to_datetime(df_zillow_zhvi["Date"], infer_datetime_format = True)


,region_name,State,Metro,Date,ZHVI,ZORI,month
0,Los Angeles County,CA,"Los Angeles-Long Beach-Anaheim, CA",2015-02-28,472084.783573,1785.665889,2015-02
1,Cook County,IL,"Chicago-Naperville-Elgin, IL-IN-WI",2015-02-28,193355.533651,1444.173886,2015-02
2,Harris County,TX,"Houston-The Woodlands-Sugar Land, TX",2015-02-28,156476.715400,1212.062984,2015-02
3,Maricopa County,AZ,"Phoenix-Mesa-Chandler, AZ",2015-02-28,211623.793977,961.637405,2015-02
4,San Diego County,CA,"San Diego-Chula Vista-Carlsbad, CA",2015-02-28,456172.515843,1688.849255,2015-02


In [100]:
#Inspect Data
df_zillow.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70724 entries, 0 to 359042
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   RegionName  70724 non-null  object        
 1   State       70724 non-null  object        
 2   Metro       70724 non-null  object        
 3   Date        70724 non-null  datetime64[ns]
 4   ZHVI        70724 non-null  float64       
 5   ZORI        70724 non-null  float64       
 6   month       70724 non-null  period[M]     
dtypes: datetime64[ns](1), float64(2), object(3), period[M](1)
memory usage: 4.3+ MB


In [121]:
#Read alpha vantage data
file_path_av = "../data/av_data.csv"
df_av = pd.read_csv(file_path_av)

#define month
df_av["date"] = pd.to_datetime(df_av["date"])
df_av["month"] = df_av["date"].dt.to_period("M")
df_av.head()

,date,spy_price,schh_price,unemployment,fed_funds,inflation,month
0,2017-01-31,199.8210,16.4257,0.047,0.000540,0.021301,2017-01
1,2017-02-28,207.6722,16.9913,0.046,0.000548,0.021301,2017-02
2,2017-03-31,207.9335,16.5194,0.044,0.000656,0.021301,2017-03
3,2017-04-30,209.9975,16.4789,0.044,0.000747,0.021301,2017-04
4,2017-05-31,212.9612,16.3817,0.044,0.000755,0.021301,2017-05


In [107]:
#Inspect Data
df_av.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          96 non-null     datetime64[ns]
 1   spy_price     96 non-null     float64       
 2   schh_price    96 non-null     float64       
 3   unemployment  96 non-null     float64       
 4   fed_funds     96 non-null     float64       
 5   inflation     84 non-null     float64       
 6   month         96 non-null     period[M]     
dtypes: datetime64[ns](1), float64(5), period[M](1)
memory usage: 5.4 KB


In [136]:
#merge three data source
df = pd.merge(df_redfin, df_zillow[["region_name", "State", "month", "ZORI", "ZHVI"]],
                     on = ["region_name", "State", "month"],
                     how = "left")
#df["month"] = pd.to_datetime(df["month"]).dt.to_period("M")
df = pd.merge(df, df_av[["month", "spy_price", "schh_price", "unemployment", "fed_funds", "inflation"]],
                     on = [ "month"],
                     how = "left")
df.head()
df.dropna(inplace=True)

,period_begin,period_end,region_name,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,median_sale_price,median_sale_price_yoy,age_of_inventory,age_of_inventory_yoy,State,month,ZORI,ZHVI,spy_price,schh_price,unemployment,fed_funds,inflation
0,2017-01-09,2017-02-05,Ulster County,34.0,0.096774,179693.7,0.060140,138.0,-11.2,NY,2017-01,1098.477609,202607.443124,199.8210,16.4257,0.047,0.000540,0.021301
6,2017-01-09,2017-02-05,Webb County,17.0,0.062500,150382.2,0.013033,70.5,14.7,TX,2017-01,870.861796,153150.841336,199.8210,16.4257,0.047,0.000540,0.021301
13,2017-01-16,2017-02-12,Ulster County,35.0,0.166667,176443.7,0.012589,138.2,-12.8,NY,2017-01,1098.477609,202607.443124,199.8210,16.4257,0.047,0.000540,0.021301
18,2017-02-13,2017-03-12,Clayton County,59.0,0.092593,109125.0,0.172757,66.2,-7.0,GA,2017-02,907.215994,93378.877456,207.6722,16.9913,0.046,0.000548,0.021301
29,2017-02-20,2017-03-19,Clayton County,59.0,-0.048387,107875.0,0.156992,63.7,-8.5,GA,2017-02,907.215994,93378.877456,207.6722,16.9913,0.046,0.000548,0.021301
